In [29]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Modifying the load_and_process_data function to aggregate the most frequent position for each player
def load_and_process_data(file_path):
    data = pd.read_excel(file_path)
    pd.set_option('display.max_rows', None)
    data = data[data["Season"]==1]

    agg_data = data.groupby('Player').agg({
        'Total Points': 'sum',
        'Goal Points': 'sum',
        'Defensive Score Points': 'sum',
        'Midfield Score': 'sum',
        'MVP Points': 'sum',
        'Date': 'count',
        'Position': lambda x: x.value_counts().index[0]  # Most frequent position
    }).rename(columns={'Date': 'Games Played'})

    agg_data['Average Points per Game'] = agg_data['Total Points'] / agg_data['Games Played']

    return agg_data

# Adjust the assign_clusters_and_prices function to add back the player position in the clustered_data
def assign_clusters_and_prices(agg_data, scaled_data):
    optimal_clusters = determine_optimal_clusters(scaled_data)
    kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', n_init=10, random_state=42)
    clusters = kmeans.fit_predict(scaled_data)
    agg_data['Cluster'] = clusters

    # Determine the average value of each cluster
    cluster_averages = agg_data.groupby('Cluster')['Average Points per Game'].mean().sort_values(ascending=False)

    # Define price values (assuming the prices are linearly decreasing from 15 to 2 for the clusters)
    prices = np.linspace(10, 2, num=optimal_clusters).round().astype(int)

    # Create a mapping of cluster to its price based on the sorted average values
    price_mapping = dict(zip(cluster_averages.index, prices))
    
    agg_data['Price (in $M)'] = agg_data['Cluster'].map(price_mapping)
    
    return agg_data

# Defining the initial functions again along with modifications

def determine_optimal_clusters(scaled_data, min_clusters=3, max_clusters=5):
    wcss = []

    for i in range(min_clusters, max_clusters+1):
        kmeans = KMeans(n_clusters=i, init='k-means++', n_init=10, random_state=42)
        kmeans.fit(scaled_data)
        wcss.append(kmeans.inertia_)
        
    # Calculate the differences between each point and its predecessor
    diffs = np.diff(wcss)
    # Return the point which has the largest difference
    optimal_clusters = np.argmin(diffs) + min_clusters + 1  # adjusting for the starting cluster number
    
    return optimal_clusters

def assign_clusters_and_prices(agg_data, scaled_data):
    optimal_clusters = determine_optimal_clusters(scaled_data)
    kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', n_init=10, random_state=42)
    clusters = kmeans.fit_predict(scaled_data)
    agg_data['Cluster'] = clusters

    # Determine the average value of each cluster
    cluster_averages = agg_data.groupby('Cluster')['Average Points per Game'].mean().sort_values(ascending=False)

    # Define price values (assuming the prices are linearly decreasing from 15 to 2 for the clusters)
    prices = np.linspace(10, 2, num=optimal_clusters).round().astype(int)

    # Create a mapping of cluster to its price based on the sorted average values
    price_mapping = dict(zip(cluster_averages.index, prices))
    
    agg_data['Price (in $M)'] = agg_data['Cluster'].map(price_mapping)
    
    return agg_data

def cluster_summary(clustered_data):
    # Grouping by cluster and aggregating data
    summary = clustered_data.groupby('Cluster').agg({
        'Total Points': ['count', 'mean'],
        'Games Played': ['mean'],
        'Average Points per Game': ['mean'],
        'Price (in $M)': 'first'  # Since all players in the same cluster have the same price
    }).reset_index()

    # Renaming columns for a cleaner look
    summary.columns = ['Cluster', 'Number of Players', 'Avg. Total Points','Average Games Played', 'Average Points/Game','Price (in $M)']
    
    return summary
file = "/workspaces/SPL/Middleput/points.xlsx"
aggregated_data = load_and_process_data(file)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(aggregated_data.drop(columns='Position'))

clustered_data = assign_clusters_and_prices(aggregated_data, scaled_data).reset_index()
clustered_summary = cluster_summary(clustered_data)
clustered_summary

Cluster  Number of Players  Avg. Total Points  Average Games Played  \
0        0                 26          20.923077              2.615385   
1        1                  7         206.857143             18.714286   
2        2                  8         157.125000             15.500000   
3        3                 23          51.173913              4.260870   

   Average Points/Game  Price (in $M)  
0             7.852564              2  
1            11.197758              7  
2            10.101056              5  
3            12.812767             10

In [22]:
clustered_data

Player  Total Points  Goal Points  Defensive Score Points  \
0            Ale (Fabri)             5            0                       0   
1            Ale (Mazzu)            17            0                       3   
2              Ale Guati            26            2                       7   
3   Alessandro Gibertini            28            2                       0   
4              Amico Gio            14            0                       0   
5           Amico Matteo             6            0                       0   
6            Andrea DeGa            19            0                       5   
7         Andrea Limonta           231            3                      72   
8       Andrea Scalambra            26            1                       8   
9      Andrea Silverstri           141           10                      38   
10                  Braz             5            0                       0   
11       Cameron McAinsh           227           42                      10   
12        Carlo Tempesta             1            0                       0   
13                 Cerro            54            4                       3   
14         Cocò (Stoppi)            28            0                       2   
15          Coyote (Cri)            68            5                       0   
16        Cri Diaz Lopez            89           11                       1   
17     Damiano Barbanera           222           52                       5   
18   Daniele (Francesco)           246           64                       6   
19     Dario (Francesco)             6            1                       0   
20            Davide Ang            35            1                       5   
21        Emilano (Dani)            67           13                       2   
22     Eugenio Gibertini            16            2                       2   
23      Fabio (Gianluca)            11            1                       2   
24      Fabrizio Limonta           151            9                      33   
25            Fede Lelli            22            2                       0   
26    Federico (Damiano)           130            8                      38   
27     Federico Paolucci           180            5                       4   
28             Francesco           101            2                      22   
29                Franco           101            1                      31   
30      Giacomo (Stoppi)            15            1                       0   
31              Gianluca            37            5                       0   
32             Gianpiero            18            0                       1   
33          Gio (Matteo)            12            1                       3   
34       Giovanni Aiello           213           12                       8   
35        Guido (Sergio)            57            1                      21   
36                 Ingra            13            0                       0   
37                Jacopo            87           15                       0   
38     Jacopo (Gianluca)            11            1                       2   
39            Leo (Cava)            20            1                       0   
40              Luca SDV            40            4                       0   
41           Luca Stoppi           245           18                      69   
42        Ludovico Righi            41            0                      19   
43          Marco Taglio            20            3                       1   
44        Matteo (Mazzu)            15            1                       0   
45     Matteo (Riccardo)           105           13                       1   
46           Matteo Virg            37            3                       0   
47              Maurizio            94           10                       3   
48                 Mazzu           177           48                       3   
49                  Omar            29            3                 